# Helpers

In [817]:
import re
from collections import Counter, defaultdict
import itertools

from tqdm.autonotebook import tqdm

/tmp/ipykernel_3939/3770615866.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [648]:
# A lot of this code is inspired from Peter Norvig's style.

def id(x): return x

def mapt(fun, it): return tuple(map(fun, it))
def first(it, pred): return next(filter(pred, it))
def parse_uint(line): return mapt(int, re.findall(r'\d+', line))
def  parse_int(line): return mapt(int, re.findall(r'-?\d+', line))

        
def read(day, process=id, split='\n'):
    with open(f'./inputs/{day}.txt') as f:
        return mapt(process, f.read().rstrip('\n').split(split))
    
def partition_into(lst: list, when, preprocess=id, postprocess=id) -> list[list]:
    res = []
    for item in map(preprocess, lst):
        if when(item): 
            yield res
            res = []
        else: 
            res.append(postprocess(item))

def partition(inp, segment_length):
    for i in range(0, len(inp), segment_length):
        yield inp[i:i+segment_length]

def windows(inp, n):
    for i in range(len(inp)):
        yield inp[i:i+n]

# Day 1

In [649]:
input = partition_into(read(1), lambda x: not x, postprocess=int)
calories = [sum(l) for  l in input]
calories.sort()
assert calories[-1] == 66487
assert sum(calories[-3:]) == 197301

# Day 2

In [650]:
scores = {
    'A X': 1+3,
    'A Y': 2+6,
    'A Z': 3+0,
    'B X': 1+0,
    'B Y': 2+3,
    'B Z': 3+6,
    'C X': 1+6,
    'C Y': 2+0,
    'C Z': 3+3,
}

In [653]:
input = read(2)

In [654]:
assert sum(map(scores.get, input)) == 10994

In [655]:
scores2 = {
    'A X': 3+0,
    'A Y': 1+3,
    'A Z': 2+6,
    'B X': 1+0,
    'B Y': 2+3,
    'B Z': 3+6,
    'C X': 2+0,
    'C Y': 3+3,
    'C Z': 1+6,
}

In [656]:
assert sum(map(scores2.get, input)) == 12526

# Day 3

In [657]:
import string

priority = dict(enumerate(string.ascii_lowercase, 1))
priority |= dict(enumerate(string.ascii_uppercase, 27))
priority = {v:k for k,v in priority.items()}

In [658]:
input = read(3, str.strip)

In [659]:
def split_half (x):
    n = len(x)//2
    return set(x[:n]), set(x[n:])

common = [x&y for x, y in map(split_half, input)]

In [660]:
sum(sum(map(priority.get, c)) for c in common)

7848

In [661]:
groups = [input[i:i+3] for i in range(0, len(input), 3)]
groups = [list(set(x)&set(y)&set(z))[0] for x, y, z in groups]
sum(map(priority.get, groups))

2616

# Day 4

In [662]:
input = read(4, parse_uint)

In [663]:
def overlap(args):
    a, b, c, d = args
    return a <= c <= d <= b or c <= a <= b <= d

In [664]:
assert sum(map(overlap, input)) == 582

In [665]:
def partial(args):
    a, b, c, d = args
    return a <= c <= b <= d or c <= a <= d <= b or overlap(args)

In [666]:
assert sum(map(partial, input)) == 893

# Day 5

In [667]:
input = read(5, lambda x: f' {x}')

In [668]:
crates = [[] for _ in range(10)]
for i in range(8):
    for j, elem in enumerate(partition(input[i], 4), 1):
        if elem.strip():
            crates[j].append(elem[2])

crates_orig = [crate[::-1] for crate in crates]

In [669]:
crates = [crate.copy() for crate in crates_orig]
for line in input[10:]:
    a, b, c = parse_int(line)
    for j in range(a):
        crates[c].append(crates[b].pop())

In [670]:
assert ''.join(crate[-1] for crate in crates[1:]) == 'CWMTGHBDW'

In [671]:
crates = [crate.copy() for crate in crates_orig]
for line in input[10:]:
    a, b, c = parse_int(line)
    crates[c].extend(crates[b][-a:])
    crates[b] = crates[b][:-a]

In [672]:
assert ''.join(crate[-1] for crate in crates[1:]) == 'SSCGWJCRB'

## Day 6

In [673]:
input = read(6)[0]

In [674]:
def solve_6(input, L):
    for i, window in enumerate(windows(input, L)):
        if len(set(window)) == L:
            return i+L

In [675]:
assert solve_6(input, 4) == 1480
assert solve_6(input, 14) == 2746

# Day 7

In [676]:
# Note: This is not the first version of the code I wrote.
# The first version was messy, and is located inside `original.py`.
# It simulated the whole filesystem as a tree, and did a depth-first-search.

In [677]:
input = read(7, split='$')[1:]

In [678]:
cwd = []
dirs = Counter()
for command in input:
    command, *output = command.strip().split()
    match command:
        case 'cd':
            match output[0]:
                case '/': 
                    cwd = ['/']
                case '..': 
                    if cwd: cwd.pop()
                case d:
                    cwd.append(d)
        case 'ls':
            if '/'.join(cwd) not in dirs:
                s = sum(int(a) for a, b in partition(output, 2) if a != 'dir')
                for i in range(len(cwd)):
                    dirs['/'.join(cwd[:i+1])] += s


In [679]:
assert sum(v for v in dirs.values() if v <= 100000) == 1306611

In [680]:
need = 30000000 - (70000000-dirs['/'])
assert min(v for v in dirs.values() if v >= need) == 13210366

# Day 8

In [681]:
import numpy as np

In [682]:
input = read(8)
# input = '''30373
# 25512
# 65332
# 33549
# 35390'''.split()


In [683]:
input = [mapt(int, line) for line in input]

In [684]:
grid = np.array(input)
h, w = grid.shape

In [685]:
td = np.maximum.accumulate(grid, axis=0)
lr = np.maximum.accumulate(grid, axis=1)
dt = np.maximum.accumulate(grid[::-1, :], axis=0)[::-1, :]
rl = np.maximum.accumulate(grid[:, ::-1], axis=1)[:, ::-1]

In [686]:
td = td[0:-2, 1:-1]
lr = lr[1:-1, 0:-2]
dt = dt[2:, 1:-1]
rl = rl[1:-1, 2:]

In [687]:
grid = grid[1:-1, 1:-1]

In [688]:
flag = grid > td
flag |= grid > dt
flag |= grid > lr
flag |= grid > rl

In [689]:
flag.sum() + h*2 + (w-2)*2

1690

In [690]:
grid = np.array(input)
h, w = grid.shape

In [421]:
mx = 0
for i in range(h):
    for j in range(w):
        col = (grid[:, j] - grid[i, j]) >= 0
        
        try:
            t = col[:i][::-1].tolist().index(True)+1
        except: 
            t = i
            
        try:
            b = col[i+1:].tolist().index(True)+1
        except:
            b = h-i-1
        
        
        row = (grid[i, :] - grid[i, j]) >= 0
        try:
            l = row[:j][::-1].tolist().index(True)+1
        except:
            l = j
            
        try:
            r = row[j+1:].tolist().index(True)+1
        except:
            r = w-j-1
        
        mx = max(mx, t*b*l*r)

In [422]:
mx

535680

In [403]:
w

99

# Day 9

In [471]:
input = '''R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2'''.split()

input = '''R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20'''.split()

In [691]:
input = read(9, split=None)

In [692]:
hr = hc = tr = tc = 0

visited = {(0, 0)}
for command in partition(input, 2):
    match command:
        case ('U', x):
            for _ in range(int(x)):
                hr += 1
                if tr < hr-1:
                    tc = hc
                    tr = hr-1
                    visited.add((tr, tc))
                # print(command, tr, tc)
        case ('D', x):
            for _ in range(int(x)):
                hr -= 1
                if tr > hr+1:
                    tc = hc
                    tr = hr+1
                    visited.add((tr, tc))
                # print(command, tr, tc)
        case ('L', x):
            for _ in range(int(x)):
                hc -= 1
                if tc > hc+1:
                    tr = hr
                    tc = hc+1
                    visited.add((tr, tc))
                # print(command, tr, tc)
        case ('R', x):
            for _ in range(int(x)):
                hc += 1
                if tc < hc-1:
                    tr = hr
                    tc = hc-1
                    visited.add((tr, tc))
                # print(command, tr, tc)


In [693]:
def vis_grid(pos):
    min_r = min(p[0] for p in pos)
    min_c = min(p[1] for p in pos)
    
    width = 50
    height = 50
    
    grid = [['.' for _ in range(width)] 
            for _ in range(height)]
    for i, (r, c) in enumerate(pos[::-1]):
        grid[min_r+r][min_c+c] = chr(ord('9') - i)
    
    return '\n'.join(''.join(line) for line in grid[::-1])

In [694]:
pos

[(15, -11),
 (15, -11),
 (15, -11),
 (15, -11),
 (15, -11),
 (15, -11),
 (15, -11),
 (15, -11),
 (15, -11),
 (14, -11)]

In [516]:
pos = [[0,0] for i in range(10)]

visited = {(0, 0)}
for (command, repeat) in partition(input, 2):
    print('X'*40)
    print(vis_grid(pos))
    # print(pos)
    for i in range(int(repeat)):
        hr, hc = pos[0]
        for j in range(1, 10):
            tr, tc = pos[j]
            moved = False
            match command:
                case 'U':
                    hr += 1
                    if tr < hr-1:
                        tc = hc
                        tr = hr-1
                        moved = True
                case 'D':
                    hr -= 1
                    if tr > hr+1:
                        tc = hc
                        tr = hr+1
                        moved = True
                case 'L':
                    hc -= 1
                    if tc > hc+1:
                        tr = hr
                        tc = hc+1
                        moved = True
                case 'R':
                    hc += 1
                    if tc < hc-1:
                        tr = hr
                        tc = hc-1
                        moved = True
            pos[j-1] = hr, hc
            hr, hc = tr, tc
            if not moved: break
        visited.add((tr, tc))
        pos[9] = (tr, tc)

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
.........................................

In [502]:

min_r = min(p[0] for p in pos)
min_c = min(p[1] for p in pos)
width = 30
height = 30

grid = [['.' for _ in range(width)] 
        for _ in range(height)]
for i, (r, c) in enumerate(pos[::-1]):
    grid[min_r+r][min_c+c] = chr(ord('9') - i)

In [500]:
print(vis_grid(pos))

..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
............................9.....................
............................8.....................
............................7.....................
............................6.....................
............................5.....................
............................4.....................
............................3.....................
............................2..

# Day 10

In [695]:
input = read(10, split='\n')

In [696]:
Xs = [1]
for inst in input:
    Xs.append(Xs[-1])
    if inst != 'noop': Xs.append(Xs[-1]+int(inst.split()[-1]))


In [697]:
sum(i*v for i, v in enumerate(Xs, 1) if (i - 20)%40 == 0)

11220

In [698]:
C = 0
X = 1
output = []
row = []

def draw():
    global row        
    c = C%40 
    row.append('#'  if X-1 <= c <= X+1 else '.')
    if len(row) == 40:
        output.append(row)
        row = []
   
for inst in input:
    draw()
    if inst == 'noop':
        C += 1
    else:
        C += 1
        draw()
        C += 1
        X += int(inst.split()[1])

In [699]:
print('\n'.join(''.join(line) for line in output))

###..####.###...##....##.####.#....#..#.
#..#....#.#..#.#..#....#.#....#....#.#..
###....#..#..#.#..#....#.###..#....##...
#..#..#...###..####....#.#....#....#.#..
#..#.#....#....#..#.#..#.#....#....#.#..
###..####.#....#..#..##..####.####.#..#.


# Day 11

In [812]:
def parse_monkey(s):
    s = s.split('\n')
    items = list(parse_uint(s[1]))
    op = s[2].split('=')[1]
    test = parse_uint(s[3])[0]
    true = parse_uint(s[4])[0]
    false = parse_uint(s[5])[0]
    
    return (items, op, test, true, false)

In [804]:
def round():
    global inspects
    for i, d in enumerate(data):
        for old in d[0]:
            inspects[i] += 1
            new = eval(d[1])
            new //= 3
            if new % d[2] == 0:
                data[d[3]][0].append(new)
            else:
                data[d[4]][0].append(new)
        d[0].clear()

In [842]:
input = read('11', split='\n\n')

In [843]:
data = [parse_monkey(s) for s in input]
inspects = [0]*len(data)
for r in range(20):
    round()

In [844]:
inspects.sort()

In [845]:
inspects

[14, 21, 302, 330, 332, 337, 344, 349]

In [846]:

div = 1
for d in data:
    div *= d[2]
div

9699690

In [847]:
def round2():
    global inspects
    for i, d in enumerate(data):
        for old in d[0]:
            inspects[i] += 1
            new = eval(d[1])
            if new % d[2] == 0:
                data[d[3]][0].append(new % div)
            else:
                data[d[4]][0].append(new % div)
        d[0].clear()

In [848]:
data = [parse_monkey(s) for s in input]
inspects = [0]*len(data)
for r in tqdm(range(10000)):
    round2()

  0%|          | 0/10000 [00:00<?, ?it/s]

In [849]:
inspects.sort()
inspects

[36384, 54845, 56502, 119552, 127607, 139769, 147556, 147854]

In [850]:
147556 * 147854

21816744824